In [83]:
import json
import re
from datetime import datetime

def cargar_json(ruta_archivo):
    with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
        datos = json.load(archivo)
    return datos

def extraer_lineas_texto(bloques):
    lineas = []
    for bloque in bloques:
        if bloque['BlockType'] == 'LINE':
            lineas.append(bloque['Text'])
    return lineas

def normalizar_texto(texto):
    mapa_caracteres = {
        'Ã\xad': 'í',
        'Ã³': 'ó',
        'Ã\xb3': 'ó',
        'Ã±': 'ñ',
        'Ã¡': 'á',
        'Ã©': 'é',
        'Ãº': 'ú',
        'Ã\xba': 'ú',
    }
    for key, value in mapa_caracteres.items():
        texto = texto.replace(key, value)
    return texto

def parsear_fecha(fecha_str):
    meses = {
        'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6,
        'Julio': 7, 'Agosto': 8, 'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12
    }
    patron = r"(\d+) de (\w+) de (\d{4}) a las (\d{2}):(\d{2}):(\d{2}) (AM|PM)"
    match = re.search(patron, fecha_str)
    if match:
        dia, mes, anio, hora, minuto, segundo, am_pm = match.groups()
        mes = meses[mes]
        hora = int(hora)
        if am_pm == 'PM' and hora != 12:
            hora += 12
        elif am_pm == 'AM' and hora == 12:
            hora = 0
        return datetime(int(anio), mes, int(dia), hora, int(minuto), int(segundo))
    return None

def extraer_info_de_lineas(lineas):
    info = {
        "numero_matricula": None,
        "fecha_impresion": None,
        "departamento": None,
        "municipio": None,
        "vereda": None,
        "estado_folio": None
    }

    patron_numero_matricula = re.compile(r"Nro Matrícula\s*:\s*(\S+)")
    patron_fecha_impresion = re.compile(r"Impreso el (.+)")
    patron_ubicacion = re.compile(r"CIRCULO REGISTRAL:.+DEPTO:\s*(\w+)\s*MUNICIPIO:\s*(\w+)\s*VEREDA:\s*(\w+)")
    patron_estado_folio_titulo = re.compile(r"ESTADO DEL FOLIO:")

    for i, linea in enumerate(lineas):
        linea_normalizada = normalizar_texto(linea)
        
        coincidencia_numero_matricula = patron_numero_matricula.search(linea_normalizada)
        if coincidencia_numero_matricula:
            info["numero_matricula"] = coincidencia_numero_matricula.group(1)

        coincidencia_fecha_impresion = patron_fecha_impresion.search(linea_normalizada)
        if coincidencia_fecha_impresion:
            fecha_str = coincidencia_fecha_impresion.group(1)
            fecha_objeto = parsear_fecha(fecha_str)
            if fecha_objeto:
                info["fecha_impresion"] = fecha_objeto.strftime("%Y-%m-%d %H:%M:%S")

        coincidencia_ubicacion = patron_ubicacion.search(linea_normalizada)
        if coincidencia_ubicacion:
            info["departamento"], info["municipio"], info["vereda"] = coincidencia_ubicacion.groups()

        coincidencia_estado_folio_titulo = patron_estado_folio_titulo.search(linea_normalizada)
        if coincidencia_estado_folio_titulo and i + 1 < len(lineas):
            info["estado_folio"] = normalizar_texto(lineas[i + 1]).strip()
    
    return info

def main(ruta_archivo):
    datos = cargar_json(ruta_archivo)
    bloques = datos['Blocks']
    lineas = extraer_lineas_texto(bloques)
    info = extraer_info_de_lineas(lineas)
    return info

# Uso
ruta_archivo = 'C:/Users/jpperez/Documents/Prueba_Practica_nlp/Prueba practica/files/040-464031-220204640254303951_pag1.json'
info = main(ruta_archivo)
print(info)

{'numero_matricula': '040-464031', 'fecha_impresion': '2022-02-04 13:16:08', 'departamento': 'ATLANTICO', 'municipio': 'BARRANQUILLA', 'vereda': 'BARRANQUILLA', 'estado_folio': 'ACTIVO'}
